# Creating the Sample Table
TODO

In [11]:
library(AnVIL)
library(xml2)
library(rentrez)
library(glue)
library(tidyverse)

# filetype in "fastq", "run"
create_sample_grid <- function(bioproject,
            semantic_name = "fastq", org = "GCP", retmax = NULL) {

        bpid <- entrez_search(db = "bioproject", term = bioproject)
    if (bpid$count == 0) {
        stop(glue::glue("Bioproject <{bioproject}> not found"))
    }

    sra_ids <- entrez_link(dbfrom = "bioproject", id = bpid$ids, db = "sra")$links$bioproject_sra
    x <- read_xml(entrez_fetch(db = "sra", id = sra_ids, rettype = "xml", retmax = retmax))
    
    runs <- xml_find_all(x, '/EXPERIMENT_PACKAGE_SET/EXPERIMENT_PACKAGE/RUN_SET/RUN')
    run_id <- xml_attr(runs, "accession")
    expref <- xml_find_all(runs, "EXPERIMENT_REF")
    experiment_id <- xml_attr(expref, "accession")
    biosample_id <- xml_attr(expref, "refname")
    # Only reporting first member in pool
    member <- xml_find_first(runs, "Pool/Member")
    sample_title <- xml_attr(member, "sample_title")
    sra_sample_id <- xml_attr(member, "accession")
    sra_File <- xml_find_first(runs, glue("./SRAFiles/SRAFile[@semantic_name=\"{semantic_name}\"]"))
    sra_File_alt <- xml_find_first(sra_File, glue("./Alternatives[@org=\"{org}\"]"))
    filename <- xml_attr(sra_File, "filename")
    sra_url <- xml_attr(sra_File_alt, "url")
    tibble(bioproject, experiment_id, biosample_id, sra_sample_id, run_id, sample_title,  filename, sra_url)
}

# PRJNA669852 -- Churchman Lab - dozens of regulatory factors
# PRJNA668299 -- Mellor Lab - Spt4
result <- create_sample_grid("PRJNA669852")

# TODO Clean up sample_id's if we are going to allow multiple assays
# Infer strain and assay type from sample_title
result %>% 
    separate(sample_title, into = c("sample_id", "assay"), sep = "_", remove = FALSE) %>%
    separate(sample_id, into = "strain", sep = "-", extra = "drop", remove = FALSE) %>%
    filter(assay == "NETseq") %>%
    relocate(sample_id) %>%
    arrange(sample_title) %>%
    rename(inputFastQ = sra_url, "entity:sample_id" = sample_id) -> out


#write.table(out, file = "~/Downloads/sample.tsv", sep = "\t", row.names = FALSE)

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.5     v stringr 1.4.0
v tidyr   1.1.4     v forcats 0.5.1
v readr   2.0.2     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter()  masks stats::filter()
x purrr::flatten() masks AnVIL::flatten()
x dplyr::lag()     masks stats::lag()


Attache Paket: 'glue'


Das folgende Objekt ist maskiert 'package:dplyr':

    collapse




In [12]:
out

entity:sample_id,bioproject,experiment_id,biosample_id,sra_sample_id,run_id,sample_title,strain,assay,filename,inputFastQ
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
bre1-1,PRJNA669852,SRX9307472,GSM4835494,SRS7534507,SRR12839969,bre1-1_NETseq,bre1,NETseq,bre1-1.fastq.gz,gs://sra-pub-src-16/SRR12839969/bre1-1.fastq.gz.1
bre1-2,PRJNA669852,SRX9307473,GSM4835495,SRS7534509,SRR12839970,bre1-2_NETseq,bre1,NETseq,bre1-2.fastq.gz,gs://sra-pub-src-16/SRR12839970/bre1-2.fastq.gz.1
bre1-3,PRJNA669852,SRX9307474,GSM4835496,SRS7534508,SRR12839971,bre1-3_NETseq,bre1,NETseq,bre1-3.fastq.gz,gs://sra-pub-src-15/SRR12839971/bre1-3.fastq.gz.1
bur2-1,PRJNA669852,SRX9307475,GSM4835497,SRS7534510,SRR12839972,bur2-1_NETseq,bur2,NETseq,bur2-1.fastq.gz,gs://sra-pub-src-16/SRR12839972/bur2-1.fastq.gz.1
bur2-2,PRJNA669852,SRX11719281,GSM5514155,SRS9749292,SRR15417434,bur2-2_NETseq,bur2,NETseq,bur2-2.fastq.gz,gs://sra-pub-src-16/SRR15417434/bur2-2.fastq.gz.1
bye1-1,PRJNA669852,SRX9307476,GSM4835498,SRS7534511,SRR12839973,bye1-1_NETseq,bye1,NETseq,bye1-1.fastq.gz,gs://sra-pub-src-17/SRR12839973/bye1-1.fastq.gz.1
bye1-2,PRJNA669852,SRX11719282,GSM5514156,SRS9749293,SRR15417435,bye1-2_NETseq,bye1,NETseq,bye1-2.fastq.gz,gs://sra-pub-src-7/SRR15417435/bye1-2.fastq.gz.1
cbc1-1,PRJNA669852,SRX9307477,GSM4835499,SRS7534512,SRR12839974,cbc1-1_NETseq,cbc1,NETseq,cbc1-1.fastq.gz,gs://sra-pub-src-18/SRR12839974/cbc1-1.fastq.gz.1
cbc1-2,PRJNA669852,SRX9307478,GSM4835500,SRS7534513,SRR12839975,cbc1-2_NETseq,cbc1,NETseq,cbc1-2.fastq.gz,gs://sra-pub-src-18/SRR12839975/cbc1-2.fastq.gz.1


In [5]:
dir(file.path(Sys.getenv("GCLOUD_SDK_PATH"), "bin"), "^(gcloud|gsutil)$")

character(0)


In [6]:
gcloud_account()

[1] "pet-110371272146592282187@broad-cpa-pipeline.iam.gserviceaccount.com"

In [7]:
gcloud_project() # billing project information


[1] "broad-cpa-pipeline"

In [8]:
avdata()

type,table,key,label,value
<chr>,<chr>,<chr>,<chr>,<chr>
other,workspace,genelist,"genelist.gff\""","\""gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/genelist.gff\"""
other,workspace,refFasta,"sacCer3.fa\""","\""gs://fc-3853b9d9-ddfd-4f0f-a505-29fa9ea4003f/sacCer3.fa\"""


In [24]:
bucket <- avbucket()
bucket
avfiles_ls()

[1] "gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02"

[1] "genelist.gff"                         
 [2] "sacCer3.fa"                           
 [3] "xwt-1.fastq.gz"                       
 [4] "0d32d7c9-9512-4db0-b6c9-6654f840de15/"
 [5] "12101555-5060-41ca-a3cf-665434207dc0/"
 [6] "15cf0b45-ea46-4123-8d08-3f6b9590c473/"
 [7] "1a3f5c55-561b-4dac-bab4-4a237574f3bd/"
 [8] "259584f6-003c-4128-bc47-14f4faf300db/"
 [9] "2e5dcc2a-e66e-4446-8f72-71df82ee3b5e/"
[10] "3021ced2-9617-4c97-a5eb-9ff8ac63d408/"
[11] "35144888-031c-49b3-9102-e9085594d153/"
[12] "3aa0a027-40bd-4ee9-9d23-194d93bd7f9b/"
[13] "3d05ec9f-1c69-42b8-9b25-e73101c83cef/"
[14] "50a3dcd6-f18c-4396-bd41-b6dbd86c0424/"
[15] "5710f9e5-759b-41e9-839f-a8a5af3efb0c/"
[16] "7c6b753c-4b80-4cef-ace4-5438279af7c9/"
[17] "836e43c3-d398-4e3c-b47f-5842352386fc/"
[18] "925461b3-bc1c-48e0-9e15-7d9a1b9d1443/"
[19] "9798b683-19cf-416c-8e84-392b4eeebc21/"
[20] "98fbe817-2868-41e5-9475-081707024361/"
[21] "a50e10b7-d638-476c-8db6-ad4306b780a4/"
[22] "ae120cb2-5a7f-470a-b8e8-ffe624d838c4/"
[23] "ae8fa1bc-bc1a-4753-b013-3db4f11d3ad4/"
[24] "b4371817-3c4a-4e5d-9e3c-564f6244348e/"
[25] "b7171dd9-f5f0-4dc9-962a-e7a44771dd5e/"
[26] "ba5bf5d6-90a3-4661-aa4a-135aef334a6f/"
[27] "cddabaa0-2b81-4b71-b475-c9ee577538bf/"
[28] "d836aca6-9b07-40a3-bbc8-0037de3f7495/"
[29] "da5b23a1-73b8-4e2f-ae18-9cadf794e0d2/"
[30] "dc4cf815-0d92-47a2-b684-04950037ab45/"
[31] "f03c7688-8a51-48da-b928-020b3d2f8a29/"
[32] "f3466e2b-c5ba-42dd-b78b-67a9817e6e8d/"
[33] "f9470c46-4a90-4aa8-89dc-8e35a0065472/"
[34] "notebooks/"

In [25]:
terra <- Terra()
terra


service: terra
tags(); use terra$<tab completion>:
# A tibble: 144 × 3
   tag      operation                     summary                               
   <chr>    <chr>                         <chr>                                 
 1 Billing  addUserToBillingProject       add user to billing project the calle…
 2 Billing  createBillingProjectFull      create billing project in FireCloud a…
 3 Billing  deleteBillingProject          delete billing project                
 4 Billing  listBillingProjectMembers     list members of billing project the c…
 5 Billing  removeUserFromBillingProject  remove user from billing project the …
 6 Entities browserDownloadEntitiesTSV    TSV file containing workspace entitie…
 7 Entities browserDownloadEntitiesTSVGet TSV file containing workspace entitie…
 8 Entities copyEntities                  Copy entities from one workspace to a…
 9 Entities deleteEntities                Bulk delete entities from a workspace 
10 Entities downloadEntitiesTSV       

In [13]:
foo <- avtable("sample")
foo

sample_id,dedup_bam,output_bam,bedgraph_pos,bedgraph_neg,inputFastQ,changepoints,logs.itemsType,logs.items
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<list>
test-1,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/45f31e25-a41c-472d-91f6-ee516e7441eb/NETseq/5e70f931-53f4-4e38-97a7-265f29b1c50f/call-BamToBedgraph/test-1.dedup.bam,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/45f31e25-a41c-472d-91f6-ee516e7441eb/NETseq/5e70f931-53f4-4e38-97a7-265f29b1c50f/call-StarAlign/test-1.aligned.bam,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/45f31e25-a41c-472d-91f6-ee516e7441eb/NETseq/5e70f931-53f4-4e38-97a7-265f29b1c50f/call-BamToBedgraph/test-1.pos.bedgraph.gz,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/45f31e25-a41c-472d-91f6-ee516e7441eb/NETseq/5e70f931-53f4-4e38-97a7-265f29b1c50f/call-BamToBedgraph/test-1.neg.bedgraph.gz,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/test-1.fastq.gz,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/fbb25897-45fd-4e50-9847-5388c7614350/netsq_to_changepoint/2f453de3-147c-4137-87fc-b26aaa69035c/call-DiscoverBreakpoints/test-1.cp.CEZINB.gff3,AttributeValue,"gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/45f31e25-a41c-472d-91f6-ee516e7441eb/NETseq/5e70f931-53f4-4e38-97a7-265f29b1c50f/call-StarAlign/test-1.Log.out , gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/45f31e25-a41c-472d-91f6-ee516e7441eb/NETseq/5e70f931-53f4-4e38-97a7-265f29b1c50f/call-StarAlign/test-1.Log.std.out , gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/45f31e25-a41c-472d-91f6-ee516e7441eb/NETseq/5e70f931-53f4-4e38-97a7-265f29b1c50f/call-StarAlign/test-1.Log.final.out, gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/45f31e25-a41c-472d-91f6-ee516e7441eb/NETseq/5e70f931-53f4-4e38-97a7-265f29b1c50f/call-BamToBedgraph/test-1.dedup.log"
wt-1,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/da5b23a1-73b8-4e2f-ae18-9cadf794e0d2/NETseq/af400abe-c96f-409d-9fa6-948f879c128b/call-BamToBedgraph/cacheCopy/wt-1.dedup.bam,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/da5b23a1-73b8-4e2f-ae18-9cadf794e0d2/NETseq/af400abe-c96f-409d-9fa6-948f879c128b/call-StarAlign/cacheCopy/wt-1.aligned.bam,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/da5b23a1-73b8-4e2f-ae18-9cadf794e0d2/NETseq/af400abe-c96f-409d-9fa6-948f879c128b/call-BamToBedgraph/cacheCopy/wt-1.pos.bedgraph.gz,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/da5b23a1-73b8-4e2f-ae18-9cadf794e0d2/NETseq/af400abe-c96f-409d-9fa6-948f879c128b/call-BamToBedgraph/cacheCopy/wt-1.neg.bedgraph.gz,gs://sra-pub-src-15/SRR12840066/wt-1.fastq.gz.1,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/50b43f2d-3c7d-4662-b652-9c44da505742/netsq_to_changepoint/84a1aef3-ddf2-4023-ae62-2caa30b6c233/call-DiscoverBreakpoints/wt-1.cp.CEZINB.gff3,AttributeValue,"gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/da5b23a1-73b8-4e2f-ae18-9cadf794e0d2/NETseq/af400abe-c96f-409d-9fa6-948f879c128b/call-StarAlign/cacheCopy/wt-1.Log.out , gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/da5b23a1-73b8-4e2f-ae18-9cadf794e0d2/NETseq/af400abe-c96f-409d-9fa6-948f879c128b/call-StarAlign/cacheCopy/wt-1.Log.std.out , gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/da5b23a1-73b8-4e2f-ae18-9cadf794e0d2/NETseq/af400abe-c96f-409d-9fa6-948f879c128b/call-StarAlign/cacheCopy/wt-1.Log.final.out, gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/da5b23a1-73b8-4e2f-ae18-9cadf794e0d2/NETseq/af400abe-c96f-409d-9fa6-948f879c128b/call-BamToBedgraph/cacheCopy/wt-1.dedup.log"
wt-2,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/50a3dcd6-f18c-4396-bd41-b6dbd86c0424/NETseq/0a094f44-e8c4-4ffd-9c73-99a8566ca3f9/call-BamToBedgraph/wt-2.dedup.bam,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/50a3dcd6-f18c-4396-bd41-b6dbd86c0424/NETseq/0a094f44-e8c4-4ffd-9c73-99a8566ca3f9/call-StarAlign/wt-2.aligned.bam,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/50a3dcd6-f18c-4396-bd41-b6dbd86c0424/NETseq/0a094f44-e8c4-4ffd-9c73-99a8566ca3f9/call-BamToBedgraph/wt-2.pos.bedgraph.gz,gs://fc-353b14e5-19da-46ef-b358-f5b85cffef02/50a3dcd6-f18c-4396-bd41-b6dbd86c0424/NETseq/0a094f44-e8c4-4ffd-9c73-99a8566ca3f9/call-BamToBedgraph/wt-2.ne

In [2]:
library(TxDb.Scerevisiae.UCSC.sacCer3.sgdGene)